In [1]:
'''
LSTM using core API- After 10,000 epochs for 50 hidden units - 85.18% accuracy for test set
'''
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf

In [3]:
training_steps = 10000
timesteps = 28
num_input = 28
bs = 128
X = tf.placeholder(tf.float32, [None, timesteps, num_input])
x = tf.unstack(X, timesteps, 1) #List of 28 timesteps of tensors of size (batch_size,28)
y_ = tf.placeholder(tf.float32, [None, 10])

In [4]:
n_hidden = 50 #Hidden units of LSTM
learning_rate = 0.001

In [5]:
'''
Used Colah's blog terminology
The final size of hidden states list will be timesteps+1 i.e. 29 (as we are using initial all_zero hidden state)
'''
with tf.device('/gpu:0'):
    hidden_states = [tf.zeros([tf.shape(X)[0],n_hidden])] #all_zero hidden state
    cell_states = [tf.zeros([tf.shape(X)[0],n_hidden])] #all_zero cell state
    ft_hidden = [tf.Variable(tf.random_normal([n_hidden,n_hidden])) for i in range(28)] 
    ft_input = [tf.Variable(tf.random_normal([28,n_hidden])) for i in range(28)]
    ft_bias = [tf.Variable(tf.random_normal([n_hidden])) for i in range(28)]
    it_hidden = [tf.Variable(tf.random_normal([n_hidden,n_hidden])) for i in range(28)] 
    it_input = [tf.Variable(tf.random_normal([28,n_hidden])) for i in range(28)]
    it_bias = [tf.Variable(tf.random_normal([n_hidden])) for i in range(28)]
    candt_hidden = [tf.Variable(tf.random_normal([n_hidden,n_hidden])) for i in range(28)] 
    candt_input = [tf.Variable(tf.random_normal([28,n_hidden])) for i in range(28)]
    candt_bias = [tf.Variable(tf.random_normal([n_hidden])) for i in range(28)]
    ot_hidden = [tf.Variable(tf.random_normal([n_hidden,n_hidden])) for i in range(28)] 
    ot_input = [tf.Variable(tf.random_normal([28,n_hidden])) for i in range(28)]
    ot_bias = [tf.Variable(tf.random_normal([n_hidden])) for i in range(28)]
    #U_hidden = [tf.Variable(tf.random_normal([n_hidden,n_hidden])) for i in range(28)]
    #W_input = [tf.Variable(tf.random_normal([28,n_hidden])) for i in range(28)]
    W_output = [tf.Variable(tf.random_normal([n_hidden,10])) for i in range(28)]
    #b = [tf.random_normal([n_hidden]) for i in range(28)]
    Y_hidden=[]
    for i in range(timesteps):
        temp = tf.matmul(hidden_states[i], ft_hidden[i])
        temp1 = tf.matmul(x[i],ft_input[i])
        ft = tf.nn.sigmoid(temp+temp1+ft_bias[i])
        temp = tf.matmul(hidden_states[i], it_hidden[i])
        temp1 = tf.matmul(x[i],it_input[i])
        it = tf.nn.sigmoid(temp+temp1+it_bias[i])
        temp = tf.matmul(hidden_states[i], candt_hidden[i])
        temp1 = tf.matmul(x[i],candt_input[i])
        candt = tf.tanh(temp+temp1+candt_bias[i])
        ct = ft*cell_states[i]+it*candt
        temp = tf.matmul(hidden_states[i], ot_hidden[i])
        temp1 = tf.matmul(x[i],ot_input[i])
        ot = tf.nn.sigmoid(temp+temp1+ot_bias[i])
        ht = ot*tf.tanh(ct)
        Y_hidden.append(tf.matmul(ht,W_output[i]))
        hidden_states.append(ht)
        cell_states.append(ct)
    final_weight = tf.Variable(tf.random_normal([n_hidden,10]))
    final_bias = tf.Variable(tf.random_normal([10]))
    # Linear activation, using rnn inner loop last output
    logits = tf.matmul(hidden_states[-1],final_weight)+final_bias
    y = tf.nn.softmax(logits)
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_))
    optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate)
    train_step = optimizer.minimize(cross_entropy)
    correct_pred = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [6]:
sess = tf.InteractiveSession()

In [7]:
tf.global_variables_initializer().run()

In [8]:
for step in range(1,training_steps+1):
    batch_xs, batch_ys = mnist.train.next_batch(bs)
    batch_xs = batch_xs.reshape((-1, 28, 28))
    sess.run(train_step, feed_dict={X: batch_xs, y_: batch_ys})
    if step%1000==0 or step==1:
        loss, acc = sess.run([cross_entropy, accuracy], feed_dict={X: batch_xs,y_: batch_ys})
        print(str(step)+" - Loss = "+str(loss)+" ,Accuracy = "+str(acc))

1 - Loss = 3.6665 ,Accuracy = 0.0859375
1000 - Loss = 1.46272 ,Accuracy = 0.460938
2000 - Loss = 1.14258 ,Accuracy = 0.601562
3000 - Loss = 0.753656 ,Accuracy = 0.734375
4000 - Loss = 0.674512 ,Accuracy = 0.8125
5000 - Loss = 0.563637 ,Accuracy = 0.757812
6000 - Loss = 0.35161 ,Accuracy = 0.851562
7000 - Loss = 0.252143 ,Accuracy = 0.875
8000 - Loss = 0.249833 ,Accuracy = 0.921875
9000 - Loss = 0.197601 ,Accuracy = 0.914062
10000 - Loss = 0.192983 ,Accuracy = 0.9375


In [9]:
print("Accuracy for test set: "+str(sess.run(accuracy, feed_dict={X: mnist.test.images.reshape([-1,timesteps,num_input]), y_: mnist.test.labels})))

Accuracy for test set: 0.8518
